# Question 3

## (a)

K-fold cross validation is implemented by:

1) Dividing dataset into k groups of len n/k.

2) Save the first fold as the validation set. 

3) Compute the MSE for all other folds, leaving out df[k], the validation set

4) Calculate the mean of all MSEs to get an estimate of the k-fold CV.

## (b)

i. Validation Set

K-fold CV has less variance as it's estimate is generated over k folds. Validation sets are a simple and lightweight computation. It is only trained once but it leaves room for high variance as a training set is not being utilized. This affects small datasets. The bias is also strong if the validation set does not properly represent the overall dataset.

ii. LOOCV

K-fold CV has better computational performance than LOOCV as n increases. The bias-variance trade off also helps propogate off variance and introducing bias as the validation set is left out. LOOCV works best on small datasets as it is more flexible and can be more precise. 

# Question 6

## (a)

In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt

default = pd.read_csv('Default.csv')
default['default'] = (default['default'].map({'Yes': 1, 'No': 0}))

X = default[['income', 'balance']]
y = default['default'] 
# model = LogisticRegression().fit(X, y)
# model.predict(X)
model = sm.OLS(X, y).fit()

In [15]:
model.coef()

{'Intercept': -11.246390742608863,
 'balance': 0.005440698914079664,
 'income': 2.149616667048059e-05}

# Balance 0.0054
# Income 2.15e-5
# Intercept -11.2464

$Y=-11.2464+3.15*10^{-5}\beta_1+0.0054\beta_2$

In [18]:
data = h2o_df
model.model_performance()

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.02138901280887861
RMSE: 0.14624983011572565
LogLoss: 0.07934050666631333
AUC: 0.9459249893470623
AUCPR: 0.50552571229206
Gini: 0.8918499786941245
Null degrees of freedom: 7996
Residual degrees of freedom: 7994
Null deviance: 2286.2564071570323
Residual deviance: 1268.9720636210154
AIC: 1274.9720636210154

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33273602746096026
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      7656  82   0.0106   (82.0/7738.0)
1      139   120  0.5367   (139.0/259.0)
Total  7795  202  0.0276   (221.0/7997.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.332736     0.520607  124
max f2                       0.117457     0.571074  233
max f0point5                 0.494298     0.586107  80
max accuracy                 0.49899      0.974365  79
max precision                0.975069     1         0
max recall                   0.0011405    1         390
max specificity              0.975069     1         0
max absolute_mcc             0.332736     0.510756  124
max min_per_class_accuracy   0.0369006    0.872189  307
max mean_per_class_accuracy  0.0346415    0.879965  311
max tns                      0.975069     7738      0
max fns                      0.975069     258       0
max fps                      3.37694e-05  7738      399
max tps                      0.0011405    259       390
max tnr                      0.975069     1         0
max fnr                      0.975069     0.996139  0
max fpr                      3.37694e-05  1         399
max tpr                      0.0011405    1         390

Gains/Lift Table: Avg response rate:  3.24 %, avg score:  3.24 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100038                   0.561985           24.3152    24.3152            0.7875           0.732737     0.7875                      0.732737            0.243243        0.243243                   2331.52   2331.52            0.241046
2        0.0200075                   0.370964           14.6663    19.4908            0.475            0.4568       0.63125                     0.594768            0.146718        0.389961                   1366.63   1849.08            0.382337
3        0.0300113                   0.288378           8.49102    15.8242            0.275            0.33119      0.5125                      0.506909            0.0849421       0.474903                   749.102   1482.42            0.459783
4        0.040015                    0.230228           6.17529    13.412             0.2              0.260482     0.434375                    0.445302            0.0617761       0.53668                    517.529   1241.2             0.513288
5        0.0500188                   0.179206           5.01742    11.7331            0.1625           0.20285      0.38                        0.396812            0.0501931       0.586873                   401.742   1073.31            0.554823
6        0.100038                    0.0681627          3.85956    7.7963             0.125            0.113936     0.2525                      0.255374            0.19305         0.779923                   285.956   679.63             0.702642
7        0.150056                    0.0379466          1.7754     5.78933            0.0575           0.0515245    0.1875            

In [6]:
def boot_fn(data, index):
    return -1

# Question 7